In [1]:
# Random Forest
import yfinance as yf
import numpy as np
import pandas as pd
from pypfopt import risk_models, EfficientFrontier
from sklearn.ensemble import RandomForestRegressor
import datetime

# tickers
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']
market_index_ticker = "^GSPC"

# start and end dates
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

# data
stock_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
market_data = yf.download(market_index_ticker, start=start_date, end=end_date)['Adj Close']

# daily returns
stock_returns = stock_data.pct_change().dropna()
market_returns = market_data.pct_change().dropna()

# store the results
results = pd.DataFrame(index=tickers, columns=['Expected Return', 'Volatility'])

for ticker in tickers:
    stock_returns_ticker = stock_returns[ticker]

    reg = RandomForestRegressor(n_estimators=100, random_state=0)
    reg.fit(np.array(market_returns).reshape(-1, 1), stock_returns_ticker)

    # predict the future returns
    future_returns = reg.predict(np.array(market_returns[-10:].mean()).reshape(-1, 1))

    # calculated the expected return
    expected_return = np.mean(future_returns)

    # calculate the volatility
    volatility = np.std(stock_returns_ticker)

    # store the results
    results.loc[ticker] = [expected_return, volatility]

# convert the results
returns = results['Expected Return'].values
cov_matrix = risk_models.sample_cov(stock_returns)

ef = EfficientFrontier(returns, cov_matrix)
ef.min_volatility()
weights = ef.clean_weights()

# calculate information
portfolio_return = np.dot(returns, np.array(list(weights.values())))
portfolio_volatility = np.sqrt(np.dot(np.array(list(weights.values())), np.dot(cov_matrix, np.array(list(weights.values())))))
sharpe_ratio = portfolio_return / portfolio_volatility

print("Weights:")
for ticker, weight in weights.items():
    print(f"{ticker}: {weight}")

print(f"Expected Return: {portfolio_return}")
print(f"Volatility: {portfolio_volatility}")
print(f"Sharpe Ratio: {sharpe_ratio}")

print("\nExpected Returns and Volatility:")
print(results)


[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  1 of 1 completed
Weights:
AAPL: 0.04937
AMD: 0.10852
AMZN: 0.01201
CCJ: 0.24064
COST: 0.02097
GOOG: 0.02141
GS: 0.0
JPM: 0.08122
LLY: 0.03196
META: 0.0314
MSFT: 0.13219
NEE: 0.02423
PFE: 0.104
SAP: 0.04747
WMT: 0.09459
Expected Return: 0.0062819225789809575
Volatility: 69.04807221090961
Sharpe Ratio: 9.097897128528974e-05

Expected Returns and Volatility:
     Expected Return Volatility
AAPL       -0.001601    0.02099
AMD        -0.000849   0.036061
AMZN        0.000425   0.022948
CCJ         0.025253   0.030372
COST        0.002132   0.015385
GOOG       -0.007467   0.020123
GS         -0.004803    0.02087
JPM        -0.001635   0.020318
LLY         0.004025   0.018964
META        0.000846   0.028245
MSFT        0.006875    0.01966
NEE         0.000745   0.017346
PFE        -0.000346   0.016644
SAP         0.001975   0.019675
WMT        -0.005503    0.01422


In [2]:
# SVM
import yfinance as yf
import numpy as np
import pandas as pd
from pypfopt import risk_models, EfficientFrontier
from sklearn.svm import SVR
import datetime

# tickers
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']
market_index_ticker = "^GSPC"

# start and end dates
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

# data
stock_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
market_data = yf.download(market_index_ticker, start=start_date, end=end_date)['Adj Close']

# daily returns
stock_returns = stock_data.pct_change().dropna()
market_returns = market_data.pct_change().dropna()

# store the results
results = pd.DataFrame(index=tickers, columns=['Expected Return', 'Volatility'])

for ticker in tickers:
    stock_returns_ticker = stock_returns[ticker]

    # use SVM
    svr = SVR()
    svr.fit(np.array(market_returns).reshape(-1, 1), stock_returns_ticker)

    future_returns = svr.predict(np.array(market_returns[-10:].mean()).reshape(-1, 1))

    # calculated the expected return
    expected_return = np.mean(future_returns)

    # calculate the volatility
    volatility = np.std(stock_returns_ticker)

    # store the results
    results.loc[ticker] = [expected_return, volatility]

# convert the results
returns = results['Expected Return'].values
cov_matrix = risk_models.sample_cov(stock_returns)

ef = EfficientFrontier(returns, cov_matrix)
ef.min_volatility()
weights = ef.clean_weights()

# calculate information
portfolio_return = np.dot(returns, np.array(list(weights.values())))
portfolio_volatility = np.sqrt(np.dot(np.array(list(weights.values())), np.dot(cov_matrix, np.array(list(weights.values())))))
sharpe_ratio = portfolio_return / portfolio_volatility

print("Weights:")
for ticker, weight in weights.items():
    print(f"{ticker}: {weight}")

print(f"Expected Return: {portfolio_return}")
print(f"Volatility: {portfolio_volatility}")
print(f"Sharpe Ratio: {sharpe_ratio}")

print("\nExpected Returns and Volatility:")
print(results)

[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  1 of 1 completed
Weights:
AAPL: 0.04935
AMD: 0.10854
AMZN: 0.01201
CCJ: 0.24067
COST: 0.02096
GOOG: 0.02142
GS: 0.0
JPM: 0.08118
LLY: 0.03194
META: 0.0314
MSFT: 0.13217
NEE: 0.02414
PFE: 0.10402
SAP: 0.04746
WMT: 0.09471
Expected Return: 0.010773252882440194
Volatility: 69.0524800799492
Sharpe Ratio: 0.00015601543738859032

Expected Returns and Volatility:
     Expected Return Volatility
AAPL        0.002969    0.02099
AMD         0.016223   0.036061
AMZN        0.031948   0.022948
CCJ         0.022261   0.030372
COST       -0.012461   0.015385
GOOG        0.002788   0.020123
GS          0.002094    0.02087
JPM         0.020769   0.020318
LLY          0.00504   0.018964
META       -0.041052   0.028245
MSFT       -0.002611    0.01966
NEE         0.001367   0.017346
PFE         0.015603   0.016644
SAP         0.031357   0.019675
WMT        -0.000328    0.01422


In [3]:
# Linear Regression
import yfinance as yf
import numpy as np
import pandas as pd
from pypfopt import risk_models, EfficientFrontier
from sklearn.linear_model import LinearRegression
import datetime

# tickers
tickers = ['AAPL', 'AMD', 'AMZN', 'CCJ', 'COST', 'GOOG', 'GS', 'JPM', 'LLY', 'META', 'MSFT', 'NEE', 'PFE', 'SAP', 'WMT']
market_index_ticker = "^GSPC"

# start and end dates
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

# data
stock_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
market_data = yf.download(market_index_ticker, start=start_date, end=end_date)['Adj Close']

# daily returns
stock_returns = stock_data.pct_change().dropna()
market_returns = market_data.pct_change().dropna()

# store the results
results = pd.DataFrame(index=tickers, columns=['Expected Return', 'Volatility'])

for ticker in tickers:
    stock_returns_ticker = stock_returns[ticker]

    # use Linear Regression
    reg = LinearRegression()
    reg.fit(np.array(market_returns).reshape(-1, 1), stock_returns_ticker)

    future_returns = reg.predict(np.array(market_returns[-10:].mean()).reshape(-1, 1))

    # calculated the expected return
    expected_return = np.mean(future_returns)

    # calculate the volatility
    volatility = np.std(stock_returns_ticker)

    # store the results
    results.loc[ticker] = [expected_return, volatility]

# convert the results
returns = results['Expected Return'].values
cov_matrix = risk_models.sample_cov(stock_returns)

ef = EfficientFrontier(returns, cov_matrix)
ef.min_volatility()
weights = ef.clean_weights()

# calculate information
portfolio_return = np.dot(returns, np.array(list(weights.values())))
portfolio_volatility = np.sqrt(np.dot(np.array(list(weights.values())), np.dot(cov_matrix, np.array(list(weights.values())))))
sharpe_ratio = portfolio_return / portfolio_volatility

print("Weights:")
for ticker, weight in weights.items():
    print(f"{ticker}: {weight}")

print(f"Expected Return: {portfolio_return}")
print(f"Volatility: {portfolio_volatility}")
print(f"Sharpe Ratio: {sharpe_ratio}")

print("\nExpected Returns and Volatility:")
print(results)

[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  1 of 1 completed
Weights:
AAPL: 0.04939
AMD: 0.10855
AMZN: 0.01201
CCJ: 0.24068
COST: 0.02096
GOOG: 0.02142
GS: 0.0
JPM: 0.08115
LLY: 0.03191
META: 0.0314
MSFT: 0.13243
NEE: 0.02418
PFE: 0.10406
SAP: 0.04738
WMT: 0.09449
Expected Return: 0.001176662682799034
Volatility: 69.0557024084596
Sharpe Ratio: 1.703932682979832e-05

Expected Returns and Volatility:
     Expected Return Volatility
AAPL        0.001511    0.02099
AMD         0.002401   0.036061
AMZN        0.000704   0.022948
CCJ         0.001384   0.030372
COST        0.000996   0.015385
GOOG        0.000927   0.020123
GS          0.000708    0.02087
JPM         0.000691   0.020318
LLY         0.001649   0.018964
META        0.000837   0.028245
MSFT        0.001327    0.01966
NEE         0.000755   0.017346
PFE         0.000385   0.016644
SAP          0.00049   0.019675
WMT         0.000709    0.01422
